In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_heatmap, plot_scatter,plot_sunburst
from core.s3 import S3AssetManager

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import List, Tuple, Optional, Sequence, Dict

In [2]:
notebook_name = "okuo_microbiologia_log_ufc"
s3 = S3AssetManager(notebook_name=notebook_name)

In [3]:
def pie_corporate(
    df: pd.DataFrame,
    label_col: str,
    value_col: Optional[str] = None,         # None => usa conteos
    *,
    title: str = "Distribución",
    colors: Optional[List[str]] = None,      # paleta; si None usa corporativa
    hole: float = 0.35,                      # 0 = pie, >0 = donut
    sort_slices: bool = False,               # mantener orden original o no
    textinfo: str = "label+percent",         # "label+percent+value", etc.
    show_legend: bool = False,
    top_n: Optional[int] = None,             # conserva top_n y agrupa resto en "Otros"
    min_share: Optional[float] = None,       # agrupa categorías con share < min_share (en 0–1)
    other_label: str = "Otros",
    center_total: bool = True,               # muestra total al centro
    value_name: str = "Valor",               # etiqueta para el hover (si value_col)
    frame: bool = True,                      # marco alrededor del plot
    save_html: Optional[str] = None,         # ruta .html

    # ---- EXTRA PARA HOVER (p. ej. prevalencias) ----
    hover_map: Optional[Dict[str, float]] = None,  # {label -> valor extra a mostrar}
    hover_label: str = "Prevalencia",
    hover_fmt: str = ".1%",                        # formato del extra (".1%", ",.2f", etc.)
) -> Tuple[go.Figure, pd.DataFrame]:
    """
    Devuelve (figura, resumen_usado).

    - Si value_col=None => cuenta ocurrencias de label_col.
    - Si value_col se provee => agrega por suma sobre label_col.
    - Puedes limitar a top_n y/o agrupar por min_share.
    - Puedes añadir un valor extra por categoría en el hover (ej. prevalencia) con hover_map.
    """

    # --- 1) Preparar datos base ---
    if value_col is None:
        data = (
            df[label_col]
            .value_counts(dropna=False)
            .rename_axis(label_col)
            .reset_index(name="value")
        )
        value_name_final = "Conteo"
    else:
        data = (
            df[[label_col, value_col]]
            .groupby(label_col, dropna=False, as_index=False)
            .sum(numeric_only=True)
            .rename(columns={value_col: "value"})
        )
        value_name_final = value_name

    # Asegurar string en labels
    data[label_col] = data[label_col].astype(str)

    # --- 2) Top-N y "Otros" por share umbral ---
    total = data["value"].sum()
    if total == 0:
        raise ValueError("Suma total de valores = 0. No se puede graficar.")

    data["share"] = data["value"] / total
    data = data.sort_values("value", ascending=False).reset_index(drop=True)

    if min_share is not None:
        small = data[data["share"] < float(min_share)]
        if not small.empty:
            keep = data[data["share"] >= float(min_share)].copy()
            row_other = pd.DataFrame({label_col: [other_label],
                                      "value": [small["value"].sum()]})
            data = pd.concat([keep[[label_col, "value"]], row_other], ignore_index=True)

    if top_n is not None and len(data) > top_n:
        keep = data.iloc[:top_n].copy()
        other = data.iloc[top_n:]["value"].sum()
        keep = pd.concat(
            [keep, pd.DataFrame({label_col: [other_label], "value": [other]})],
            ignore_index=True
        )
        data = keep

    # Recalcular share final y ordenar si corresponde
    total = data["value"].sum()
    data["share"] = data["value"] / total
    if sort_slices:
        data = data.sort_values("value", ascending=False).reset_index(drop=True)

    labels = data[label_col].tolist()
    values = data["value"].tolist()

    # --- 3) Colores corporativos por defecto ---
    default_colors = [
        "#1A494C", "#17877D", "#94AF92", "#F6B27A", "#F18F01",
        "#E4572E", "#6C757D", "#343A40", "#A3CED0",
    ]
    palette = colors or default_colors
    palette = (palette * ((len(labels) // len(palette)) + 1))[: len(labels)]

    # --- 4) EXTRA EN HOVER (p. ej. prevalencia por categoría) ---
    extra_vals = None
    if hover_map is not None:
        # Alinea el valor extra al orden actual de labels
        extra_vals = [hover_map.get(str(lbl), float("nan")) for lbl in labels]

    hover_lines = [
        "<b>%{label}</b>",
        f"{value_name_final}: %{{value:,}}",
        "Participación: %{percent:.1%}",
    ]
    if extra_vals is not None:
        # Inserta el extra como segunda línea
        hover_lines.insert(1, f"{hover_label}: %{{customdata[0]:{hover_fmt}}}")
    hover_tmpl = "<br>".join(hover_lines) + "<extra></extra>"

    # --- 5) Figura ---
    fig = go.Figure(
        data=[
            go.Pie(
                labels=labels,
                values=values,
                hole=hole,
                sort=False,  # controlamos el orden arriba
                marker=dict(colors=palette, line=dict(color="#FFFFFF", width=1)),
                textinfo=textinfo,
                textposition="inside",
                customdata=None if extra_vals is None else np.c_[extra_vals],
                hovertemplate=hover_tmpl,
            )
        ]
    )

    # --- 6) Layout corporativo ---
    layout_kwargs = dict(
        title=dict(text=title, x=0.5, font=dict(size=20)),
        font=dict(family="Inter, 'Helvetica Neue', Arial, sans-serif", size=12, color="#FFFFFF"),
        showlegend=show_legend,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=20, r=20, t=70 if show_legend else 60, b=20),
    )
    if frame:
        layout_kwargs["shapes"] = [dict(
            type="rect", xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="#E5E7EB", width=2),
            fillcolor="rgba(0,0,0,0)",
        )]
    fig.update_layout(**layout_kwargs)

    # Texto al centro con total
    if center_total:
        fig.update_layout(annotations=[dict(
            text=f"<b>Total</b><br>{total:,.0f}",
            x=0.5, y=0.5, showarrow=False,
            font=dict(size=14, color="#0B3B3C")
        )])

    # Guardar si se solicita
    if save_html:
        fig.write_html(save_html, include_plotlyjs="cdn")

    # Resumen que usó la gráfica (útil para tablas/exports)
    summary = data[[label_col, "value", "share"]].copy()
    summary["percent"] = (summary["share"] * 100).round(2)

    return fig, summary

In [4]:
def _apply_explicit_order(df: pd.DataFrame, col: str, order: list | None):
    """Convierte una columna a categoría ordenada si se pasa 'order'."""
    if order:
        df = df.copy()
        df[col] = pd.Categorical(df[col].astype(str), categories=order, ordered=True)
    return df

def two_pies_subplots(
    *,
    # --- Pie IZQUIERDA ---
    df_left: pd.DataFrame,
    label_left: str,
    title_left: str,
    prev_map_left: dict | None = None,      # p.ej. {"Animal":0.31, ...}
    # --- Pie DERECHA ---
    df_right: pd.DataFrame,
    label_right: str,
    title_right: str,
    prev_map_right: dict | None = None,
    # --- Opciones de orden interno ---
    sort_slices_left: bool = False,
    sort_slices_right: bool = False,
    explicit_order_left: list | None = None,    # p.ej. ["Animal","Vegetal","Otra"]
    explicit_order_right: list | None = None,
    # --- Layout / formato ---
    swap: bool = False,                     # True => intercambia izquierda<->derecha
    hole: float = 0.35,
    width: int = 1100,
    height: int = 520,
    overall_title: str = "<b>Distribuciones en MP</b>",
    horizontal_spacing: float = 0.08,
    show_legend: bool = False,
    save_html: str | None = None,
):
    """
    Devuelve un subplot 1x2 con dos donuts corporativos usando 'pie_corporate'.
    - Puedes invertir lados con 'swap=True'
    - Puedes ordenar slices por valor o por orden explícito
    - Muestra total al centro de cada donut
    """

    # Copias seguras y orden explícito (si aplica)
    _left  = _apply_explicit_order(df_left,  label_left,  explicit_order_left)
    _right = _apply_explicit_order(df_right, label_right, explicit_order_right)

    # Construir cada pie con nuestra función corporativa
    fig_left,  res_left  = pie_corporate(
        df=_left,
        label_col=label_left,
        value_col=None,                   # tamaño por conteo
        title=title_left,
        hover_map=prev_map_left,
        hover_label="Prevalencia",
        hover_fmt=".1%",
        sort_slices=sort_slices_left,
        center_total=False,               # lo añadimos manual en el subplot
        show_legend=show_legend,
        hole=hole,
    )

    fig_right, res_right = pie_corporate(
        df=_right,
        label_col=label_right,
        value_col=None,
        title=title_right,
        hover_map=prev_map_right,
        hover_label="Prevalencia",
        hover_fmt=".1%",
        sort_slices=sort_slices_right,
        center_total=False,
        show_legend=show_legend,
        hole=hole,
    )

    # Subplot 1x2 de tipo 'domain'
    sub = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "domain"}, {"type": "domain"}]],
        horizontal_spacing=horizontal_spacing,
        subplot_titles=(title_left, title_right) if not swap else (title_right, title_left),
    )

    # Añadir trazas (con intercambio si swap=True)
    if not swap:
        sub.add_trace(fig_left.data[0],  row=1, col=1)
        sub.add_trace(fig_right.data[0], row=1, col=2)
        total_left, total_right = res_left["value"].sum(), res_right["value"].sum()
        center_left_x, center_right_x = 0.185, 0.812  # ajusta si cambias spacing
    else:
        sub.add_trace(fig_right.data[0], row=1, col=1)
        sub.add_trace(fig_left.data[0],  row=1, col=2)
        total_left, total_right = res_right["value"].sum(), res_left["value"].sum()
        center_left_x, center_right_x = -0.6, -0.

    # Estilo corporativo + marco
    sub.update_layout(
        width=width, height=height,
        title=dict(text=overall_title, x=0.5, font=dict(size=22)),
        font=dict(family="Inter, 'Helvetica Neue', Arial, sans-serif", size=12, color="#0B3B3C"),
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=30, r=30, t=90, b=30),
        shapes=[dict(
            type="rect", xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="#E5E7EB", width=2), fillcolor="rgba(0,0,0,0)"
        )],
        showlegend=show_legend,
    )

    # Totales al centro
    sub.add_annotation(
        x=center_left_x, y=0.5, xref="paper", yref="paper", showarrow=False,
        text=f"<b>Total Analysis</b><br>{total_left:,.0f}", font=dict(size=14, color="#0B3B3C")
    )
    sub.add_annotation(
        x=center_right_x, y=0.5, xref="paper", yref="paper", showarrow=False,
        text=f"<b>Total Analysis</b><br>{total_right:,.0f}", font=dict(size=14, color="#0B3B3C")
    )

    if save_html:
        sub.write_html(save_html, include_plotlyjs="cdn")

    return sub


In [13]:
df = s3.read_excel("raw/bios/PLATAFORMA_INOCUIDAD_JULIO.xlsx", sheet_name="base de datos")
df.columns = [x.strip().lower() for x in df.columns]

df["resultado_num"] = pd.to_numeric(df["resultado"], errors='coerce')
df.loc[df["resultado_num"] <= 0, "resultado_num"] = np.nan

df["resultado_num"] = df["resultado_num"].fillna(1)
df["resultado_log"] = np.log10(df["resultado_num"])
df["have_micro"] = np.where(df["resultado_num"]==1, False, True)

df["date"] = pd.to_datetime(df["yyyy-mm-dd del análisis"])
df = df[df["date"].between('2025-06-30', '2025-09-01')]
df["aw"] = pd.to_numeric(df["aw"], errors='coerce')
df["kilogramos"] = pd.to_numeric(df["kilogramos"], errors='coerce')


mapea_proceso = {
    "Materia Prima": "Materia Prima",
    "Producto Terminado": "Producto Terminado",
    "Superficie": "Proceso Interno",
    "Ambiente": "Proceso Interno",
    "Restaurante/Casino": "Proceso Interno",
    "Agua": "Proceso Interno",
}


df["process_etapa"] = df["tipo de muestra"].map(mapea_proceso)

#TODO: por muestra se realiza un análisis por microorganismos por día
ref_unique_raw = ['identificación', 'muestra', 'microorganismo', 'date']
df = df.drop_duplicates(ref_unique_raw)

otra_fuente= 'Misceláneas'
mapping_origen_harina = {
    "Harina De Galleta Dulce": otra_fuente,
    "Harina De Galleta Salada": otra_fuente,
    "Cookie Meal": otra_fuente,

    "Pescado.Hna. 65": "Animal",
    "Pescado.Hna. 56": "Animal",
    "Pescado.Hna. 58": "Animal",
    "Tilapia, Hna": "Animal",
    "Camaron, Hna": "Animal",

    "Carne Hueso, Hna": "Animal",
    "Carne Hueso, Hna (A)": "Animal",
    "Carne, Hna": "Animal",
    "Harina de Cordero": "Animal",
    "Sebo": "Animal",
    "Hemoglobina": "Animal",
    "Plasma Porcino": "Animal",
    "Plasma Bovino": "Animal",

    "Plumas, Hna.": "Animal",
    "Plumas BD, Hna": "Animal",
    "Pollo, Hna.": "Animal",
    "Pollo VisC. Hna": "Animal",
    "Pollo Viscera Hna Bd": "Animal",
    "Bites de Pollo": "Animal",

    "Bites de Arveja": "Vegetal",
    "Bites de Zanahoria": "Vegetal",
    "Bites de Manzana": "Vegetal",
    "Alfalfa": "Vegetal",
    "Palma, Aceite": "Vegetal",

    "Premez. Base": otra_fuente,
    "Saborizante liquído": otra_fuente,
    "Saborizante polvo": otra_fuente,
}
df["origen_harina"] = df["muestra"].map(mapping_origen_harina).fillna(otra_fuente)

df["microorganismo"] = df["microorganismo"].replace({
    'Clostridium Sulfito reductor': 'Clostridium Sulfitoreductor'
})

import string

cats = sorted(df["planta"].dropna().unique())  # orden estable
labels = [f"Centro Productivo {l}" for l in string.ascii_uppercase[:len(cats)]]

map_planta = dict(zip(cats, labels))

df["planta"] = df["planta"].map(map_planta)
map_planta  # para ver el diccionario


{'Bogotá': 'Centro Productivo A',
 'Bucaramanga': 'Centro Productivo B',
 'Buga': 'Centro Productivo C',
 'Cartago': 'Centro Productivo D',
 'Ciénaga de Oro': 'Centro Productivo E',
 'Envigado': 'Centro Productivo F',
 'Itagüi': 'Centro Productivo G',
 'Mosquera': 'Centro Productivo H',
 'Neiva': 'Centro Productivo I'}

In [14]:
df_mp = df[df["tipo de muestra"] == 'Materia Prima']

In [15]:
def compute_(df, cols):
    df_group = df.groupby(cols).agg(
        sample_n=('identificación', 'nunique'),
        analysis_n=('identificación', 'count'),
        resultado_num=("resultado_num", "mean"),
        prev=("have_micro", "mean"),
        positive_case=("have_micro", "sum"),

        aw=('aw', 'mean'),
        kg=("kilogramos", "sum"),

    ).reset_index()
    df_group["resultado_log"] = np.log10(df_group["resultado_num"])
    return df_group

In [16]:
mp_origin = compute_(df=df_mp, cols=["origen_harina"])
mp_origin

,origen_harina,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log
0,Animal,231,1300,3091.378045,0.311538,405,0.389223,33409331.0,3.490152
1,Misceláneas,46,255,2877.544392,0.345098,88,0.500000,4714068.0,3.459022
2,Vegetal,14,79,10391.730725,0.316456,25,0.210000,636250.0,4.016688


In [17]:
mean_map_micro_mp = (
    df_mp.groupby("microorganismo")["resultado_log"]
         .mean()
         .rename_axis("microorganismo")
         .reset_index()
         .assign(microorganismo=lambda d: d["microorganismo"].astype(str))
         .set_index("microorganismo")["resultado_log"]
         .to_dict()
)

mean_map_origen_mp = (
    df_mp.groupby("origen_harina")["resultado_log"]
         .mean()
         .rename_axis("origen_harina")
         .reset_index()
         .assign(origen_harina=lambda d: d["origen_harina"].astype(str))
         .set_index("origen_harina")["resultado_log"]
         .to_dict()
)

fig = two_pies_subplots(
    df_right=df_mp, label_right="origen_harina",
    title_right="<b>porcentaje de análisis por origen de harina</b>",
    prev_map_right=mean_map_origen_mp,   # pasamos el mapa (lo usaremos en hover)

    df_left=df_mp, label_left="microorganismo",
    title_left="<b>porcentaje de análisis por microorganismo</b>",
    prev_map_left=mean_map_micro_mp,     # pasamos el mapa (lo usaremos en hover)

    sort_slices_left=True,
    explicit_order_right=None,
    swap=False,
    width=1200, height=520,
    overall_title="<b>Distribución de análisis en MP</b>",
)

for tr in fig.data:
    labels = list(tr.labels)
    # Intentamos mapear primero por microorganismo, luego por origen
    vals_mic = [mean_map_micro_mp.get(lbl, np.nan) for lbl in labels]
    vals_org = [mean_map_origen_mp.get(lbl, np.nan) for lbl in labels]
    use_vals = vals_mic if np.sum(np.isfinite(vals_mic)) >= np.sum(np.isfinite(vals_org)) else vals_org

    tr.customdata = np.c_[use_vals]
    tr.hovertemplate = (
        "<b>%{label}</b><br>"
        "Participación: %{percent:.1%}<br>"
        "Log(UFC/g) (promedio): %{customdata[0]:.2f}"
        "<extra></extra>"
    )

fig.show()
s3.save_plotly_html(fig, "pies_origen_micro_mp.html")

In [18]:
mp_planta = compute_(df=df_mp, cols=["planta"])
mp_origin_plant = compute_(df=df_mp, cols=["planta", "origen_harina"])
mp_planta_microorganismo = compute_(df=df_mp, cols=["planta", "microorganismo"])
mp_planta_origen_micro = compute_(df=df_mp, cols=["planta", "origen_harina", "microorganismo"])

In [21]:
levels = ["planta", "proveedor", "origen_harina"]
mp_planta_prov_origen_micro = compute_(df=df_mp, cols=levels)
title_level = ' → '.join(levels)

f = plot_sunburst(
    
     df=mp_planta_prov_origen_micro,
    path_cols=levels,
    metric_col="resultado_log",
    title=f"<b>Log(UFC/g) en MP distribuida por {title_level}</b>",
     width=600, height=600,
)
f.show()
s3.save_plotly_html(f, "subplot_distribuciones_mp.html")

# PT

In [22]:
df_pt = df[df["tipo de muestra"] == "Producto Terminado"]
df_pt

,empresa,planta,yyyy-mm-dd del análisis,identificación,tipo de muestra,muestra,proveedor,presentación,tipo de alimento,especie,...,observaciones,aw,kilogramos,zonificación,resultado_num,resultado_log,have_micro,date,process_etapa,origen_harina
0,Finca S.A.S,Centro Productivo C,2025-07-01,499 // 282550,Producto Terminado,Ringo Croquetas,Finca Buga,Bulto,Extruido,Mascotas,...,NaN,0.52,29995.0,Z0,1.0,0.000000,False,2025-07-01,Producto Terminado,Misceláneas
1,Finca S.A.S,Centro Productivo C,2025-07-01,499 // 282550,Producto Terminado,Ringo Croquetas,Finca Buga,Bulto,Extruido,Mascotas,...,100% Penicillium,0.52,29995.0,Z0,10.0,1.000000,True,2025-07-01,Producto Terminado,Misceláneas
2,Finca S.A.S,Centro Productivo C,2025-07-01,499 // 282550,Producto Terminado,Ringo Croquetas,Finca Buga,Bulto,Extruido,Mascotas,...,NaN,0.52,29995.0,Z0,3100.0,3.491362,True,2025-07-01,Producto Terminado,Misceláneas
3,Finca S.A.S,Centro Productivo C,2025-07-01,499 // 282550,Producto Terminado,Ringo Croquetas,Finca Buga,Bulto,Extruido,Mascotas,...,NaN,0.52,29995.0,Z0,1.0,0.000000,False,2025-07-01,Producto Terminado,Misceláneas
4,Finca S.A.S,Centro Productivo C,2025-07-01,499 // 282550,Producto Terminado,Ringo Croquetas,Finca Buga,Bulto,Extruido,Mascotas,...,NaN,0.52,29995.0,Z0,10.0,1.000000,True,2025-07-01,Producto Terminado,Misceláneas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,Contegral S.A.S,Centro Productivo I,2025-07-29,MIC696,Producto Terminado,C. Tilapias Iniciacion,Contegral S.A.S,Bulto,Harina,Acuacultura,...,20507282 - Nucleo Reversion,0.38,11520.0,Z0,4.0,0.602060,True,2025-07-29,Producto Terminado,Misceláneas
3127,Contegral S.A.S,Centro Productivo I,2025-07-29,MIC696,Producto Terminado,C. Tilapias Iniciacion,Contegral S.A.S,Bulto,Harina,Acuacultura,...,20507282 - Nucleo Reversion,0.38,11520.0,Z0,49400.0,4.693727,True,2025-07-29,Producto Terminado,Misceláneas
3128,Contegral S.A.S,Centro Productivo I,2025-07-29,MIC696,Producto Terminado,C. Tilapias Iniciacion,Contegral S.A.S,Bulto,Harina,Acuacultura,...,20507282 - Nucleo Reversion,0.38,11520.0,Z0,1.0,0.000000,False,2025-07-29,Producto Terminado,Misceláneas
3129,Contegral S.A.S,Centro Productivo I,2025-07-29,MIC696,Producto Terminado,C. Tilapias Iniciacion,Contegral S.A.S,Bulto,Harina,Acuacultura,...,20507282 - Nucleo Reversion,0.38,11520.0,Z0,80.0,1.903090,True,2025-07-29,Producto Terminado,Misceláneas


In [23]:

mean_map_micro = (
    df_pt.groupby("microorganismo")["resultado_log"]
         .mean()
         .rename_axis("microorganismo")
         .reset_index()
         .assign(microorganismo=lambda d: d["microorganismo"].astype(str))
         .set_index("microorganismo")["resultado_log"]
         .to_dict()
)

mean_map_tipo = (
    df_pt.groupby("tipo de alimento")["resultado_log"]
         .mean()
         .rename_axis("tipo de alimento")
         .reset_index()
         .assign(**{"tipo de alimento": lambda d: d["tipo de alimento"].astype(str)})
         .set_index("tipo de alimento")["resultado_log"]
         .to_dict()
)

# --- 2) Construir los dos pies (puedes pasar cualquier dict en prev_map_*) ---
fig = two_pies_subplots(
    df_right=df_pt, label_right="tipo de alimento",
    title_right="<b>Porcentaje de análisis por tipo de alimento</b>",
    prev_map_right=mean_map_tipo,        # << usamos promedio resultado_log

    df_left=df_pt, label_left="microorganismo",
    title_left="<b>Porcentaje de análisis por microorganismo</b>",
    prev_map_left=mean_map_micro,        # << usamos promedio resultado_log

    sort_slices_left=True,
    explicit_order_right=None,
    swap=False,
    width=1200, height=520,
    overall_title="<b>Porcentaje de análisis en PT</b>",
    #save_html=f"{ROOT_IMAGEN}/subplot_distribuciones_pt.html",
)

# --- 3) Reemplazar el hover de ambos donuts para mostrar resultado_log promedio ---
for tr in fig.data:
    # Detectar a qué mapa pertenece cada traza según sus labels
    labels = list(tr.labels)
    vals_left  = [mean_map_micro.get(lbl, np.nan) for lbl in labels]
    vals_right = [mean_map_tipo.get(lbl,  np.nan) for lbl in labels]
    # elegimos el mapa que tenga más aciertos
    left_hits  = sum(np.isfinite(v) for v in vals_left)
    right_hits = sum(np.isfinite(v) for v in vals_right)
    vals = vals_left if left_hits >= right_hits else vals_right

    tr.customdata = np.c_[vals]  # columna única con promedio resultado_log
    tr.hovertemplate = (
        "<b>%{label}</b><br>"
        "Participación: %{percent:.1%}<br>"
        "Log(UFC/g): %{customdata[0]:.2f}"
        "<extra></extra>"
    )

fig.show()
s3.save_plotly_html(fig, f"subplot_distribuciones_pt.html")

In [25]:
levels = ["planta", 'tipo de alimento',  'muestra']
pt_planta_microorganismo = compute_(df=df_pt, cols=levels)
title_level = ' → '.join(levels)

f = plot_sunburst(
    
      df=pt_planta_microorganismo,
      path_cols=levels,
      value_col='analysis_n',
      metric_col="resultado_log",
      title=f"<b>Log(UFC/g) en PT distribuida por {title_level}</b>",
      width=600, height=600,
)
f.show()
s3.save_plotly_html(f, "sunburst_planta_muestra_presentacion_micro_pt_result.html")

# AGUA PROCESO

In [26]:
cond_agua = df["tipo de muestra"] == 'Agua'
cond_muestra = df["muestra"] == 'Agua Adicionada a Proceso '
df_agua_process = df[cond_agua & cond_muestra]
df_agua_process

,empresa,planta,yyyy-mm-dd del análisis,identificación,tipo de muestra,muestra,proveedor,presentación,tipo de alimento,especie,...,observaciones,aw,kilogramos,zonificación,resultado_num,resultado_log,have_micro,date,process_etapa,origen_harina
316,Finca S.A.S,Centro Productivo C,2025-07-15,566,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,NaN,NaN,NaN,Z0,200.0,2.30103,True,2025-07-15,Proceso Interno,Misceláneas
317,Finca S.A.S,Centro Productivo C,2025-07-15,566,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,NaN,NaN,NaN,Z0,1.0,0.00000,False,2025-07-15,Proceso Interno,Misceláneas
318,Finca S.A.S,Centro Productivo C,2025-07-15,566,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,NaN,NaN,NaN,Z0,2.0,0.30103,True,2025-07-15,Proceso Interno,Misceláneas
337,Finca S.A.S,Centro Productivo C,2025-07-23,571,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,AGUA GRIFO CARTAGO VM01501014,NaN,NaN,Z0,1.0,0.00000,False,2025-07-23,Proceso Interno,Misceláneas
338,Finca S.A.S,Centro Productivo C,2025-07-23,571,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,AGUA GRIFO CARTAGO VM01501014,NaN,NaN,Z0,1.0,0.00000,False,2025-07-23,Proceso Interno,Misceláneas
339,Finca S.A.S,Centro Productivo C,2025-07-23,571,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,AGUA GRIFO CARTAGO VM01501014,NaN,NaN,Z0,1.0,0.00000,False,2025-07-23,Proceso Interno,Misceláneas
478,Finca S.A.S,Centro Productivo C,2025-07-30,595,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,Agua Adicionada a Proceso,NaN,NaN,Z0,1.0,0.00000,False,2025-07-30,Proceso Interno,Misceláneas
479,Finca S.A.S,Centro Productivo C,2025-07-30,595,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,Agua Adicionada a Proceso,NaN,NaN,Z0,1.0,0.00000,False,2025-07-30,Proceso Interno,Misceláneas
480,Finca S.A.S,Centro Productivo C,2025-07-30,595,Agua,Agua Adicionada a Proceso,Finca Buga,No aplica,No aplica,Agua adicionada a Proceso,...,Agua Adicionada a Proceso,NaN,NaN,Z0,1.0,0.00000,False,2025-07-30,Proceso Interno,Misceláneas
796,Contegral S.A.S,Centro Productivo D,2025-07-08,1106//Ingreso mezcladora,Agua,Agua Adicionada a Proceso,Contegral Cartago,No aplica,No aplica,Agua adicionada a Proceso,...,NaN,NaN,NaN,Z0,1.0,0.00000,False,2025-07-08,Proceso Interno,Misceláneas


In [27]:
levels = ["planta"]
agua_planta = compute_(df=df_agua_process, cols=levels)

In [29]:
levels = ["planta", 'microorganismo']
agua_process_planta_microorganismo = compute_(df=df_agua_process, cols=levels)
title_level = ' → '.join(levels)

f = plot_sunburst(
        df=agua_process_planta_microorganismo,
        path_cols=levels,
        value_col='analysis_n',
        metric_col="resultado_log",
        title=f"<b>Log(UFC/g) en Agua del proceso distribuida por {title_level}</b>",
         width=600, height=600,
)
f.show()
s3.save_plotly_html(f, "sunburst_agua_process_results.html")


# Superficies

In [30]:
cond_super = df["tipo de muestra"] == 'Superficie'
sup_process = [
    'Ensacadora',
    'Empaque',
    'Secador',
   'Postengrase',
    'Mezcladora',
   'Ensacadora',
   'Enfriador '
]
especie = ['Superficie Producción', 'Manipulador Producción']
cond_muestra = df["muestra"].isin(sup_process)
cond_especie = df["especie"].isin(especie)
df_super_process = df[cond_super & cond_especie]

In [31]:
levels = ["planta"]
superficie_planta = compute_(df=df_super_process, cols=levels)

In [33]:
levels = ["planta", 'microorganismo']
super_process_planta_microorganismo = compute_(df=df_super_process, cols=levels)
title_level = ' → '.join(levels)

f = plot_sunburst(
        df=super_process_planta_microorganismo,
        path_cols=levels,
        value_col='analysis_n',
        metric_col="resultado_log",
        title=f"<b>Log(UFC/g) en Superficies del proceso distribuida por {title_level}</b>",
        width=600, height=600,
)
f.show()
s3.save_plotly_html(f, "sunburst_superficie_process_result.html")



In [35]:
super_process_planta_microorganismo.groupby(["planta", "microorganismo"]).agg(
    prom=("resultado_log", "mean")
)

prom
planta              microorganismo              
Centro Productivo C Coliformes Totales  1.700415
                    E.Coli              0.000000
Centro Productivo D Coliformes Totales  2.057920
                    E.Coli              0.000000
                    Salmonella          0.000000
Centro Productivo F Coliformes Totales  0.000000
                    E.Coli              0.000000
                    Salmonella          0.000000
Centro Productivo G Coliformes Totales  0.000000
                    E.Coli              0.000000
                    Salmonella          0.000000
Centro Productivo I Coliformes Totales  0.000000
                    E.Coli              0.000000
                    Salmonella          0.000000

In [36]:
levels = ["planta"]
pt_planta = compute_(df=df_pt, cols=levels)

In [37]:
levels = ["planta"]
agua_planta = compute_(df=df_agua_process, cols=levels)

# UNION

In [38]:
pt_planta["etapa"] = "pt"
mp_planta["etapa"] = "mp"
agua_planta["etapa"] = "agua_process"
superficie_planta["etapa"] = "superficie"
d_planta = pd.concat([mp_planta,pt_planta, agua_planta, superficie_planta])
d_planta

,planta,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,etapa
0,Centro Productivo A,6,36,15697.888889,0.250000,9,0.284000,846250.0,4.195841,mp
1,Centro Productivo B,3,18,2146.924242,0.388889,7,NaN,0.0,3.331817,mp
2,Centro Productivo C,29,166,2399.066265,0.439759,73,0.450303,3864118.0,3.380042,mp
3,Centro Productivo D,146,784,1647.860398,0.339286,266,NaN,22412000.0,3.216920,mp
4,Centro Productivo E,11,66,3992.904959,0.257576,17,0.354000,74830.0,3.601289,mp
5,Centro Productivo F,19,104,16308.093531,0.336538,35,NaN,0.0,4.212403,mp
6,Centro Productivo G,10,60,15403.922727,0.350000,21,NaN,0.0,4.187631,mp
7,Centro Productivo H,11,66,1384.530303,0.257576,17,0.502000,1579140.0,3.141302,mp
8,Centro Productivo I,56,334,911.341317,0.218563,73,0.373694,9983311.0,2.959681,mp
0,Centro Productivo A,11,66,27029.575758,0.318182,21,0.650909,264000.0,4.431839,pt


In [39]:
def group_global(df, cols):
    d_planta_group = df.groupby(cols).agg(
        analysis_n=('analysis_n', 'sum'),
        positive_case=('positive_case', 'sum'),
        resultado_num=('resultado_num', 'sum')
    ).reset_index()
    d_planta_group["prev"] =d_planta_group['positive_case']/d_planta_group['analysis_n']
    d_planta_group["resultado_log"] = np.log10(d_planta_group["resultado_num"])
    return d_planta_group


In [40]:
d_planta_group = group_global(d_planta, ['planta', 'etapa'])

In [41]:
CORPORATE_COLORS_ = [
'#17877D',
'#94AF92',
'#F6B27A',
'#F18F01',
'#E4572E',
'#6C757D',
]

In [42]:
fig_23 = plot_heatmap(
    df=d_planta_group,
    x_col="etapa",
    y_col="planta",
    value_col="resultado_log",
    x_order= ["mp", "agua_process", "superficie",  "pt"],
    show_secondary_labels=False,
    title="Log(UFC/g) promedio por etapa y planta",
    width=1100, height=350,
    colorscale=CORPORATE_COLORS_,
)
fig_23.show()
s3.save_plotly_html(fig_23, "resultado_log_etapa_planta_.html")

In [43]:
pt_planta_microorganismo = compute_(df=df_pt, cols=["planta", "microorganismo"])

In [44]:
pt_planta_microorganismo["etapa"] = "pt"
mp_planta_microorganismo["etapa"] = "mp"
agua_process_planta_microorganismo["etapa"] = "agua_process"
super_process_planta_microorganismo["etapa"] = "superficie"
d_planta_micro = pd.concat([pt_planta_microorganismo,mp_planta_microorganismo, agua_process_planta_microorganismo, super_process_planta_microorganismo])

d_planta_micro_group = group_global(d_planta_micro, ['planta', 'etapa', "microorganismo"])
d_planta_micro_group

,planta,etapa,microorganismo,analysis_n,positive_case,resultado_num,prev,resultado_log
0,Centro Productivo A,mp,Clostridium Sulfitoreductor,6,2,8.500000,0.333333,0.929419
1,Centro Productivo A,mp,Coliformes Totales,6,1,52.500000,0.166667,1.720159
2,Centro Productivo A,mp,E.Coli,6,0,1.000000,0.000000,0.000000
3,Centro Productivo A,mp,Hongos y Levadura,6,3,55.500000,0.500000,1.744293
4,Centro Productivo A,mp,Mesofilos,6,3,94068.833333,0.500000,4.973446
...,...,...,...,...,...,...,...,...
117,Centro Productivo I,pt,Mesofilos,22,22,16469.090909,1.000000,4.216670
118,Centro Productivo I,pt,Salmonella,22,0,1.000000,0.000000,0.000000
119,Centro Productivo I,superficie,Coliformes Totales,3,0,1.000000,0.000000,0.000000
120,Centro Productivo I,superficie,E.Coli,3,0,1.000000,0.000000,0.000000


In [45]:
d_planta_group

,planta,etapa,analysis_n,positive_case,resultado_num,prev,resultado_log
0,Centro Productivo A,mp,36,9,15697.888889,0.250000,4.195841
1,Centro Productivo A,pt,66,21,27029.575758,0.318182,4.431839
2,Centro Productivo B,mp,18,7,2146.924242,0.388889,3.331817
3,Centro Productivo C,agua_process,9,2,23.222222,0.222222,1.365904
4,Centro Productivo C,mp,166,73,2399.066265,0.439759,3.380042
5,Centro Productivo C,pt,251,108,2367.103586,0.430279,3.374217
6,Centro Productivo C,superficie,12,5,25.583333,0.416667,1.407957
7,Centro Productivo D,agua_process,3,1,2.000000,0.333333,0.301030
8,Centro Productivo D,mp,784,266,1647.860398,0.339286,3.216920
9,Centro Productivo D,pt,92,29,1777.888389,0.315217,3.249904


In [71]:

for pl in d_planta_micro_group["planta"].unique():
    df_ = d_planta_micro_group[d_planta_micro_group["planta"] == pl]

    f = plot_heatmap(
            df=df_,
            x_col="etapa",
            y_col="microorganismo",
            value_col="resultado_log",
            x_order= ["mp", "agua_process", "superficie",  "pt"],
            secondary_col="analysis_n",
            secondary_aggfunc="count",
            show_secondary_labels=False,
            title=f"Log(UFC/g) promedio por etapa en {pl}",
            width=1100, height=350,
            colorscale=CORPORATE_COLORS_
            )
    f.show()
    name = f"resultado_log_etapa_{pl}_.html"
    print(name)
    s3.save_plotly_html(f, f"resultado_log_etapa_{pl}_.html")


resultado_log_etapa_Centro Productivo A_.html


resultado_log_etapa_Centro Productivo B_.html


resultado_log_etapa_Centro Productivo C_.html


resultado_log_etapa_Centro Productivo D_.html


resultado_log_etapa_Centro Productivo E_.html


resultado_log_etapa_Centro Productivo F_.html


resultado_log_etapa_Centro Productivo G_.html


resultado_log_etapa_Centro Productivo H_.html


resultado_log_etapa_Centro Productivo I_.html


# Comparativa entre plantas

In [47]:

#, df_pt, df_agua_process
df_total = pd.concat([df_mp])
grupos = compute_(df_total, cols=["planta", "microorganismo"])
grupos

,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log
0,Centro Productivo A,Clostridium Sulfitoreductor,6,6,8.500000,0.333333,2,0.284000,144850.0,0.929419
1,Centro Productivo A,Coliformes Totales,6,6,52.500000,0.166667,1,0.284000,144850.0,1.720159
2,Centro Productivo A,E.Coli,6,6,1.000000,0.000000,0,0.284000,144850.0,0.000000
3,Centro Productivo A,Hongos y Levadura,6,6,55.500000,0.500000,3,0.284000,144850.0,1.744293
4,Centro Productivo A,Mesofilos,6,6,94068.833333,0.500000,3,0.284000,144850.0,4.973446
5,Centro Productivo A,Salmonella,6,6,1.000000,0.000000,0,0.284000,122000.0,0.000000
6,Centro Productivo B,Clostridium Sulfitoreductor,3,3,10.666667,0.333333,1,NaN,0.0,1.028029
7,Centro Productivo B,Coliformes Totales,3,3,1.000000,0.000000,0,NaN,0.0,0.000000
8,Centro Productivo B,E.Coli,3,3,1.000000,0.000000,0,NaN,0.0,0.000000
9,Centro Productivo B,Hongos y Levadura,3,3,454.545455,1.000000,3,NaN,0.0,2.657577


In [48]:

r = grupos["resultado_log"].rank(pct=True, method="average")   # 0..1
grupos["decil_num"] = (
    np.ceil(r * 10).clip(1, 10).astype("Int64")           # 1..10
)
grupos["decil"] = pd.Categorical(grupos["decil_num"].map(lambda x: f"D{x}" if pd.notna(x) else np.nan),
                             categories=[f"D{i}" for i in range(1, 11)], ordered=True)

grupos["decil"].value_counts()


decil
D2     17
D5      6
D8      6
D10     6
D6      5
D7      5
D9      5
D4      4
D1      0
D3      0
Name: count, dtype: int64

In [49]:
grupos.groupby(["decil"], observed=True).agg(
    minimo=('resultado_log', "min"),
    maximo=('resultado_log', "max"),
)

,minimo,maximo
decil,,
D2,0.000000,0.000000
D4,0.130929,0.862532
D5,0.929419,1.409975
D6,1.474481,1.709609
D7,1.720159,1.807781
D8,1.942008,2.657577
D9,2.737273,4.009770
D10,4.093888,4.996313


In [50]:
grupos[grupos["decil"] == "D4"]

,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,decil_num,decil
20,Centro Productivo D,E.Coli,143,143,7.286713,0.006993,1,NaN,4156000.0,0.862532,4,D4
31,Centro Productivo F,Coliformes Totales,17,17,4.352941,0.176471,3,NaN,0.0,0.638783,4,D4
45,Centro Productivo H,Hongos y Levadura,11,11,6.181818,0.272727,3,0.502000,263190.0,0.791116,4,D4
49,Centro Productivo I,Coliformes Totales,54,54,1.351852,0.018519,1,0.372778,1608008.0,0.130929,4,D4


In [51]:
grupos[grupos["decil"] == "D9"]

,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,decil_num,decil
13,Centro Productivo C,Coliformes Totales,28,28,750.857143,0.500000,14,0.450357,651985.0,2.875557,9,D9
22,Centro Productivo D,Mesofilos,122,122,10227.508197,0.868852,106,NaN,3442000.0,4.009770,9,D9
33,Centro Productivo F,Hongos y Levadura,17,17,546.101604,0.352941,6,NaN,0.0,2.737273,9,D9
46,Centro Productivo H,Mesofilos,11,11,8253.818182,0.818182,9,0.502000,263190.0,3.916655,9,D9
52,Centro Productivo I,Mesofilos,54,54,5531.648148,0.833333,45,0.372778,1608008.0,3.742855,9,D9


In [52]:
grupos[grupos["decil"] == "D9"]["planta"].nunique()


5

In [53]:
grupos[grupos["decil"] == "D9"]


,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,decil_num,decil
13,Centro Productivo C,Coliformes Totales,28,28,750.857143,0.500000,14,0.450357,651985.0,2.875557,9,D9
22,Centro Productivo D,Mesofilos,122,122,10227.508197,0.868852,106,NaN,3442000.0,4.009770,9,D9
33,Centro Productivo F,Hongos y Levadura,17,17,546.101604,0.352941,6,NaN,0.0,2.737273,9,D9
46,Centro Productivo H,Mesofilos,11,11,8253.818182,0.818182,9,0.502000,263190.0,3.916655,9,D9
52,Centro Productivo I,Mesofilos,54,54,5531.648148,0.833333,45,0.372778,1608008.0,3.742855,9,D9


In [54]:
grupos[grupos["decil"] == "D7"]

,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,decil_num,decil
1,Centro Productivo A,Coliformes Totales,6,6,52.500000,0.166667,1,0.284000,144850.0,1.720159,7,D7
3,Centro Productivo A,Hongos y Levadura,6,6,55.500000,0.500000,3,0.284000,144850.0,1.744293,7,D7
12,Centro Productivo C,Clostridium Sulfitoreductor,28,28,55.464286,0.535714,15,0.450357,651985.0,1.744013,7,D7
30,Centro Productivo F,Clostridium Sulfitoreductor,17,17,59.882353,0.529412,9,NaN,0.0,1.777299,7,D7
39,Centro Productivo G,Hongos y Levadura,10,10,64.236364,0.400000,4,NaN,0.0,1.807781,7,D7


In [55]:
name_plant = set(df["planta"])
name_plant

{'Centro Productivo A',
 'Centro Productivo B',
 'Centro Productivo C',
 'Centro Productivo D',
 'Centro Productivo E',
 'Centro Productivo F',
 'Centro Productivo G',
 'Centro Productivo H',
 'Centro Productivo I'}

In [56]:
name_plant - set(grupos[grupos["decil"] == "D7"]["planta"])

{'Centro Productivo B',
 'Centro Productivo D',
 'Centro Productivo E',
 'Centro Productivo H',
 'Centro Productivo I'}

In [57]:


def add_ntiles(
    df: pd.DataFrame,
    value_col: str,
    n: int = 10,
    *,
    by: Optional[Sequence[str]] = None,   # p.ej. ["categoria"] si quieres cortar dentro de cada grupo
    method: str = "average",              # rank method: "average", "min", "max", "first", "dense"
    ascending: bool = True,               # True = valores bajos en ntile 1
    prefix: Optional[str] = None,         # None -> "D" si n=10, "Q" si n=5, en otro caso "G"
    out_name: Optional[str] = None        # prefijo para los nombres de salida
) -> pd.DataFrame:
    """
    Añade dos columnas: <out_name>_num (Int64: 1..n) y <out_name> (Categorical ordenada).
    Respeta NaNs del value_col.
    """

    if out_name is None:
        out_name = f"{value_col}_{n}tiles"
    if prefix is None:
        prefix = {10: "D", 5: "Q"}.get(n, "G")

    def _compute(g: pd.DataFrame) -> pd.DataFrame:
        s = g[value_col]
        # ranking porcentual 0..1 (NaNs quedan como NaN)
        r = s.rank(pct=True, method=method, ascending=ascending)
        # 1..n (Int64), conservando NaN donde aplique
        nt_num = np.ceil(r * n).clip(1, n)
        nt_num = pd.Series(nt_num.where(s.notna(), np.nan), index=g.index).astype("Int64")

        cats = [f"{prefix}{i}" for i in range(1, n + 1)]
        nt_cat = pd.Categorical(
            nt_num.map(lambda x: f"{prefix}{x}" if pd.notna(x) else np.nan),
            categories=cats,
            ordered=True
        )

        g[out_name + "_num"] = nt_num
        g[out_name] = nt_cat
        return g

    if by is None:
        return _compute(df.copy())
    else:
        return df.groupby(list(by), group_keys=False).apply(_compute)


In [58]:
# Deciles globales (D1..D10)
grupos = add_ntiles(grupos, "resultado_log", n=10)
grupos["resultado_log_10tiles"].value_counts()




resultado_log_10tiles
D2     17
D5      6
D8      6
D10     6
D6      5
D7      5
D9      5
D4      4
D1      0
D3      0
Name: count, dtype: int64

In [59]:
grupos[grupos["resultado_log_10tiles"] == "D10"]

,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,decil_num,decil,resultado_log_10tiles_num,resultado_log_10tiles
4,Centro Productivo A,Mesofilos,6,6,94068.833333,0.500000,3,0.284000,144850.0,4.973446,10,D10,10,D10
10,Centro Productivo B,Mesofilos,3,3,12413.333333,1.000000,3,NaN,0.0,4.093888,10,D10,10,D10
16,Centro Productivo C,Mesofilos,28,28,13339.357143,0.928571,26,0.450357,651985.0,4.125135,10,D10,10,D10
28,Centro Productivo E,Mesofilos,11,11,23675.454545,1.000000,11,NaN,0.0,4.374298,10,D10,10,D10
34,Centro Productivo F,Mesofilos,17,17,99154.705882,1.000000,17,NaN,0.0,4.996313,10,D10,10,D10
40,Centro Productivo G,Mesofilos,10,10,92259.000000,1.000000,10,NaN,0.0,4.965009,10,D10,10,D10


## SIMILARIDAD PREV

In [60]:
## PREV similares por planta
df_total = pd.concat([df_pt])
gr = compute_(df_total, cols=["planta", "microorganismo"])
gr_prev = add_ntiles(gr, "resultado_log", n=5)
gr_prev

,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,resultado_log_5tiles_num,resultado_log_5tiles
0,Centro Productivo A,Clostridium Sulfitoreductor,11,11,5.454545,0.090909,1,0.650909,44000.0,0.736759,2,Q2
1,Centro Productivo A,Coliformes Totales,11,11,14.454545,0.181818,2,0.650909,44000.0,1.160004,3,Q3
2,Centro Productivo A,E.Coli,11,11,1.000000,0.000000,0,0.650909,44000.0,0.000000,1,Q1
3,Centro Productivo A,Hongos y Levadura,11,11,10.000000,0.727273,8,0.650909,44000.0,1.000000,2,Q2
4,Centro Productivo A,Mesofilos,11,11,162145.545455,0.909091,10,0.650909,44000.0,5.209905,5,Q5
5,Centro Productivo A,Salmonella,11,11,1.000000,0.000000,0,0.650909,44000.0,0.000000,1,Q1
6,Centro Productivo C,Clostridium Sulfitoreductor,42,42,32.904762,0.714286,30,0.616053,1309985.0,1.517259,4,Q4
7,Centro Productivo C,Coliformes Totales,42,42,20.476190,0.285714,12,0.616053,1309985.0,1.311249,3,Q3
8,Centro Productivo C,E.Coli,42,42,1.000000,0.000000,0,0.616053,1309985.0,0.000000,1,Q1
9,Centro Productivo C,Hongos y Levadura,41,41,27.243902,0.585366,24,0.620270,1278035.0,1.435269,3,Q3


In [61]:
gr_prev.groupby("resultado_log_5tiles", observed=True).agg(
    min_=('resultado_log', "min"),
    max_=('resultado_log', "max"),
    count=('resultado_log', 'count'),)

,min_,max_,count
resultado_log_5tiles,,,
Q1,0.000000,0.000000,16
Q2,0.215600,1.000000,3
Q3,1.153815,1.440766,9
Q4,1.478435,2.939207,10
Q5,3.201714,5.209905,10


In [62]:
name_plant

{'Centro Productivo A',
 'Centro Productivo B',
 'Centro Productivo C',
 'Centro Productivo D',
 'Centro Productivo E',
 'Centro Productivo F',
 'Centro Productivo G',
 'Centro Productivo H',
 'Centro Productivo I'}

In [63]:
df_total = pd.concat([df_pt])
gr = compute_(df_total, cols=["planta", "microorganismo"])
gr_prev = add_ntiles(gr, "resultado_log", n=5)
gr_prev

,planta,microorganismo,sample_n,analysis_n,resultado_num,prev,positive_case,aw,kg,resultado_log,resultado_log_5tiles_num,resultado_log_5tiles
0,Centro Productivo A,Clostridium Sulfitoreductor,11,11,5.454545,0.090909,1,0.650909,44000.0,0.736759,2,Q2
1,Centro Productivo A,Coliformes Totales,11,11,14.454545,0.181818,2,0.650909,44000.0,1.160004,3,Q3
2,Centro Productivo A,E.Coli,11,11,1.000000,0.000000,0,0.650909,44000.0,0.000000,1,Q1
3,Centro Productivo A,Hongos y Levadura,11,11,10.000000,0.727273,8,0.650909,44000.0,1.000000,2,Q2
4,Centro Productivo A,Mesofilos,11,11,162145.545455,0.909091,10,0.650909,44000.0,5.209905,5,Q5
5,Centro Productivo A,Salmonella,11,11,1.000000,0.000000,0,0.650909,44000.0,0.000000,1,Q1
6,Centro Productivo C,Clostridium Sulfitoreductor,42,42,32.904762,0.714286,30,0.616053,1309985.0,1.517259,4,Q4
7,Centro Productivo C,Coliformes Totales,42,42,20.476190,0.285714,12,0.616053,1309985.0,1.311249,3,Q3
8,Centro Productivo C,E.Coli,42,42,1.000000,0.000000,0,0.616053,1309985.0,0.000000,1,Q1
9,Centro Productivo C,Hongos y Levadura,41,41,27.243902,0.585366,24,0.620270,1278035.0,1.435269,3,Q3


In [64]:
gr_prev.groupby("resultado_log_5tiles", observed=True).agg(
    min_=('resultado_log', "min"),
    max_=('resultado_log', "max"),
    count=('resultado_log', 'count'),)

,min_,max_,count
resultado_log_5tiles,,,
Q1,0.000000,0.000000,16
Q2,0.215600,1.000000,3
Q3,1.153815,1.440766,9
Q4,1.478435,2.939207,10
Q5,3.201714,5.209905,10


In [65]:
gr_prev[gr_prev["resultado_log_5tiles"] == "Q5"]["planta"].unique()

array(['Centro Productivo A', 'Centro Productivo C',
       'Centro Productivo D', 'Centro Productivo E',
       'Centro Productivo F', 'Centro Productivo G',
       'Centro Productivo H', 'Centro Productivo I'], dtype=object)

In [66]:
gr_prev[gr_prev["resultado_log_5tiles"] == "Q5"]["microorganismo"].unique()

array(['Mesofilos', 'Hongos y Levadura'], dtype=object)

In [67]:
name_plant-set(gr_prev[gr_prev["resultado_log_5tiles"] == "Q2"]["planta"])

{'Centro Productivo B',
 'Centro Productivo C',
 'Centro Productivo D',
 'Centro Productivo E',
 'Centro Productivo F',
 'Centro Productivo G',
 'Centro Productivo I'}

In [68]:
log_plant_specie = compute_(df_pt, cols=["planta", "especie", "microorganismo"])
log_plant_specie["kg"] = pd.to_numeric(log_plant_specie["kg"])

In [69]:
fig = plot_scatter(
    df=log_plant_specie,
    x_col="aw",                      # Antes: x="aw"
    y_col="resultado_log",           # Antes: y="resultado_log"
    group_col="microorganismo",      # Antes: hue="microorganismo"
    
    # Datos Extra al pasar el mouse
    hover_data_cols=["planta", "analysis_n", "especie"], 
    
    # Títulos
    title="Log(UFC/g) por Actividad de Agua y especie de Microorganismo",
    x_title="aW", 
    y_title="Log(UFC/g)",
    
    # Estilos
    width=1000,
    height=500,
    marker_size=12,                  # Puntos un poco más grandes para legibilidad
    marker_opacity=0.8,
    #marker_colors=CORPORATE_COLORS,  # Usar tu paleta
    
)

fig.show()
s3.save_plotly_html(fig, f"log_aw_micro.html")

In [70]:
fig = plot_scatter(
    df=log_plant_specie[log_plant_specie["kg"] <=117600.0],
    x_col="kg",                      # Antes: x="aw"
    y_col="resultado_log",           # Antes: y="resultado_log"
    group_col="microorganismo",      # Antes: hue="microorganismo"
    
    # Datos Extra al pasar el mouse
    hover_data_cols=["planta", "analysis_n", "especie"], 
    
    # Títulos
    title="Log(UFC/g) por kilogramo producido y especie de Microorganismo",
    x_title="Producción (Kg)", 
    y_title="Log(UFC/g)",
    
    # Estilos
    width=1000,
    height=500,
    marker_size=12,                  # Puntos un poco más grandes para legibilidad
    marker_opacity=0.8,
    #marker_colors=CORPORATE_COLORS,  # Usar tu paleta
    
)

fig.show()
s3.save_plotly_html(fig, f"log_kg_micro.html")